In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD
import random

lemmatizer = WordNetLemmatizer()
nltk.download("popular")

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/drive/MyDrive/chatbot/Dataset.json').read()
intents = json.loads(data_file)
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

# create our training data
training = []
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in an attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in the current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)  # Specify dtype=object to allow for variable-length sub-arrays

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons,
# and 3rd output layer contains the number of neurons equal to the number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model using updated SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5')

print("Model created")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

78 documents
23 classes ['1-2 orang', '3-5 orang', '6-10 orang', 'Farewell', 'Flysheet', 'Harga sewa Peralatan', 'Hutan', 'Kompor', 'Layanan Pelanggan', 'Matras', 'Menu Metode 1', 'Menu Metode 2', 'Menu Metode 3', 'Menu Metode 4', 'Menu Metode 5', 'Metode Pembayaran', 'Nesting/Cooking set', 'Pantai', 'Perkenalan', 'Pesan Peralatan', 'Start', 'Tas Carier', 'Tenda']
67 unique lemmatized words ['1', '1-2', '10', '2', '3', '3-5', '4', '5', '6', '6-10', '7', '8', '9', 'alat', 'assalamualaikum', 'baik', 'cara', 'carier', 'cooking', 'cukup', 'di', 'flysheet', 'fungsi', 'funsi', 'hai', 'halo', 'harga', 'hello', 'hi', 'hutan', 'kasih', 'kompor', 'kontak', 'layanan', 'malam', 'matras', 'metode', 'mulai', 'nesting', 'nesting/cooking', 'ok', 'oke', 'orang', 'pagi', 'pantai', 'pelanggan', 'pembayaran', 'penggunaan', 'penyewaan', 'peralatan', 'perlengkapan', 'pesan', 'rekomedasi', 'rekomendasi', 'sekian', 'selamat', 'set', 'sewa', 'siang', 'sore', 'ta', 'tegal', 'tempat', 'tenda', 'tentang', 'terima

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
!cp labels.pkl '/content/drive/MyDrive/chatbot'

In [10]:
!cp model.h5 '/content/drive/MyDrive/chatbot'

In [11]:
!cp texts.pkl '/content/drive/MyDrive/chatbot'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
